In [3]:
import os

#import keras
from keras.models import load_model

import numpy as np
import cv2
import random
from time import sleep
from PIL import Image

#このpgmファイルの格納フォルダ
this_pgm_folder = os.getcwd()

In [14]:
#画像格納先フォルダ
#※注意：画像ファイル名は、 「"正解ラベル名"+"_"+"～.～"」 とすること。
img_folder = os.path.join(this_pgm_folder , 'image')

#モデルの格納先フォルダ
model_folder = os.path.join(this_pgm_folder , 'model')

#ラベル
cifar10_labels = np.array([
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck'])

#画像ファイルとラベルの辞書
img_label_dict = {}
img_folder_flist = os.listdir(img_folder)
for file_name in img_folder_flist:
    img_label_dict[file_name] = file_name.split('_')[0]

#選択肢の候補（cifar10のラベル、取り込む画像に存在するラベル、その他）
choice_candidate = list(cifar10_labels) + ['Sandwich Man(Comedian)'
                                              , 'shark'
                                              , 'monkey'
                                              , 'Pico Taro']

#使用するモデル（この順にモデルを使用するため、難度を考慮し、出力候補が少ない順にする。）
model_list = [
    'model_0_2.hdf5'
    , 'model_3_5.hdf5'
    , 'model_4_7.hdf5'
    , 'model_0_1_8_9.hdf5'
    , 'model_2_3_4_5_6_7.hdf5'
    , 'model_0_1_2_3_4_5_6_7_8_9.hdf5'
]

#読み込む祭のサイズ
size = (32 , 32)

#画像のリサイズ、正規化
#引数：対象の画像（cv2オブジェクト）、リサイズ後のサイズ（タプル）
def img_resize_normalize(img , size):
    #リサイズ
    img_resize = cv2.resize(img , size)
    
    #正規化
    img_resize_normalize = img_resize / 255
    
    return img_resize_normalize

#問題選択肢の生成
#引数：選択肢の数、正解ラベル、選択肢の候補（ラベルのリスト）
#戻り値：{1:ラベル名1 , 2:ラベル名2 , …}
def gen_answer_choice(choice_num , ans_label , choice_candidate):
    #第三引数の選択肢の候補から正解を除外
    #※参照渡しを防ぐためにスライス記法
    choice_candidate_rem_answer = choice_candidate[:]
    choice_candidate_rem_answer.remove(ans_label)
    
    #選択肢に出すラベル
    choice_labels_tmp = [ans_label] + random.sample(choice_candidate_rem_answer , choice_num - 1)
    
    choice_labels = {}
    for_max = len(choice_labels_tmp)
    for i in range(for_max):
        choice_labels[i] = random.choice(choice_labels_tmp)
        choice_labels_tmp.remove(choice_labels[i])
    
    return choice_labels

#問題作成・判定
#引数：使用するモデルのリスト、判定対象画像ファイル名、正解ラベル、選択肢の数、選択肢の候補リスト
def create_question_judge(model_list , given_img_file , ans_label , choice_num , choice_candidate):
    #画像オブジェクト生成
    os.chdir(img_folder)
    img = cv2.imread(given_img_file)
    
    #画像のリサイズ、標準化
    img = img_resize_normalize(img , size)
    
    #選択肢
    choice_labels = gen_answer_choice(choice_num , ans_label , choice_candidate)
    
    #モデルの格納先
    os.chdir(model_folder)
    
    #終了するときは「1」にする
    end_flg = 0
    
    for i in range(len(model_list)):  
        if end_flg == 0:
            #モデルのファイル名
            model_file = model_list[i]

            #モデルの出力対象のリスト（cifar10のラベルリストのインデックスのリスト）
            gen_labels_num_tmp = model_file.replace('model_' , '').replace('.hdf5' , '').split('_')
            gen_labels_num = list(map(lambda x : int(x) , gen_labels_num_tmp))
            
            #モデルの出力対象のリスト（ラベル名のリスト）
            #（参考）モデルの出力は、cifar10_labelsの順になっている。
            #　　　　(例：dog,catが出力対象なら、pred[*][0]がdogの確率、pred[*][1]がcatの確率)
            model_labels_list = np.array(cifar10_labels)[gen_labels_num]

            #予測
            print('画像認識中...')
            model = load_model(model_file)
            preds = model.predict(np.array([img]) , verbose = 0)

            #予測結果の出力テキスト
            text_pred = ''
            for j in range(len(preds[0])):
                text_pred += '\n' + model_labels_list[j] + '：' + str(round(preds[0][j] * 100 , 1) ) + '%'

            #選択肢の出力テキスト
            text_candidate = ''
            for number , label in choice_labels.items():
                text_candidate += str(number) + ':' + str(label) + ' '

            #出力
            print('*' * 10 + '【モデル' + str(i+1) + '/' + str(len(model_list)) + '個目の認識結果】' + '*' * 10)
            print(text_pred)
            print('')
            print('【選択肢】')
            print(text_candidate)
            
            input_ok_flg = 0
            while(input_ok_flg == 0):
                answer = input('回答(番号) :')
                if answer in (str(i) for i in range(choice_num)):
                    input_ok_flg = 1

            #待ち時間(秒)
            print('・・・')
            sleep(1)

            if choice_labels[int(answer)] == ans_label:
                print('正解!!')
                end_flg = 1
            else :
                print('残念...')

                input_ok_flg = 0
                while(input_ok_flg == 0):
                    continue_flg = input('続けますか？ y/n：')
                    if continue_flg in ('y' , 'n'):
                        input_ok_flg = 1

                if continue_flg == 'n':
                    end_flg = 1
                    print('答えは...「' + ans_label + '」でした。')
                    
            print('*' * 47)
            
        

In [15]:
#処理実行
print('◆' * 50)
print('問題：')
print('ある画像を複数のモデルで認識した結果を出します。')
print('何の画像か答えてください。')
print()

sleep(1)

#終了するとき「1」にする
end_flg = 0

for i in range(len(img_label_dict)):
    if end_flg == 0:
        #画像ファイル、正解ラベル
        (given_img_file , ans_label) = list(img_label_dict.items())[i]

        print('【第' + str(i+1) + '問 開始】' + '+' * 50)

        create_question_judge(model_list , given_img_file , ans_label , len(model_list) + 1 , choice_candidate)
        Image.open(os.path.join(img_folder , given_img_file)).show()
        
        print('【第' + str(i+1) + '問 終了】' + '+' * 50)
        
        input_ok_flg = 0
        while(input_ok_flg == 0):
            continue_flg = input('次の問題に進みますか？ y/n：')
            if continue_flg in ('y' , 'n'):
                input_ok_flg = 1

        if continue_flg == 'n':
            end_flg = 1

print()
print('【終了】')
print('◆' * 50)

◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆
問題：
ある画像を複数のモデルで認識した結果を出します。
何の画像か答えてください。

【第1問 開始】++++++++++++++++++++++++++++++++++++++++++++++++++
画像認識中...
**********【モデル1/6個目の認識結果】**********

airplane：59.4%
bird：40.6%

【選択肢】
0:deer 1:dog 2:ship 3:Pico Taro 4:Sandwich Man(Comedian) 5:horse 6:automobile 
回答(番号) :3
・・・
残念...
続けますか？ y/n：y
***********************************************
画像認識中...
**********【モデル2/6個目の認識結果】**********

cat：6.3%
dog：93.7%

【選択肢】
0:deer 1:dog 2:ship 3:Pico Taro 4:Sandwich Man(Comedian) 5:horse 6:automobile 
回答(番号) :1
・・・
正解!!
***********************************************
【第1問 終了】++++++++++++++++++++++++++++++++++++++++++++++++++
次の問題に進みますか？ y/n：n

【終了】
◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆
